In [1]:
%%capture
%cd ..
# Capture disables output
# cd to import data correctly
# Only run this cell once, bc it will cd up another directory

In [2]:
import folium
import pandas as pd
import json

In [3]:
!curl https://cfc-gcfd-default-rtdb.firebaseio.com/geoJsonV2/counties.json -o shape_files/county_v2.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 53542  100 53542    0     0   425k      0 --:--:-- --:--:-- --:--:--  428k


In [4]:
wic_total = pd.read_csv('final_jsons/wic_participation_total.csv')

In [5]:
with open('final_jsons/metaData.json') as f:
    bins = json.load(f)
wic_total_bins = bins['data_bins']['natural_breaks']['wic_participation_total_data']['total']

In [6]:
wic_county_str = wic_total.fips.apply(lambda x: str(x)[2:])
wic_total['county_str'] = wic_county_str
wic_total.head()

,fips,NAME,race_amer_indian_or_alaskan_native,race_asian,race_black,race_native_hawaii_or_pacific_islander,race_white,race_multiracial,total,hispanic_or_latino,county_str
0,17001,ADAMS,4,6,172,8,697,95,780,44,001
1,17003,ALEXANDER,0,0,85,1,85,15,156,4,003
2,17005,BOND,0,1,29,0,347,12,364,3,005
3,17007,BOONE,0,1,27,2,714,5,741,455,007
4,17009,BROWN,0,1,1,0,51,1,52,3,009


In [7]:
IL_lat_long = [40.63, -89.40]
county_shp = 'shape_files/county_v2.json'

m = folium.Map(
    location=IL_lat_long,
    zoom_start=6
)

folium.Choropleth(
    geo_data=county_shp,
    name = 'county_choropleth',
    data = wic_total,
    columns = ['county_str', 'total'],
    key_on = 'feature.properties.COUNTY',
    fill_color = "YlGn",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    bins = wic_total_bins
).add_to(m)

folium.LayerControl().add_to(m)

m

In [8]:
# Label overlap in Folium appears to be an unsolved issue: https://github.com/python-visualization/folium/issues/1052 & https://stackoverflow.com/questions/61441415/python-folium-choropleth-map-legend-quantiles-overlap